In [12]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [13]:
data = pd.read_csv("McAlindon_Big.csv")
print(data.shape)
data.head()

(15428, 43)


,ID,Race,age,Income,bmi,treat,sex,nsaid,Occupation,retire,...,stiff.4,physi.7,stiff.7,WeatherDate,avgdewpt,avgtemp,maxtemp,mintemp,precip,press
0,178,White Non-Hispanic,65,"<$14,999",24.429492,1,1,1,Retired,2.0,...,2.0,18.0,0.0,57,56.0,70.0,86.0,53.0,0.0,27.62
1,178,White Non-Hispanic,65,"<$14,999",24.429492,1,1,1,Retired,2.0,...,2.0,18.0,0.0,58,58.0,74.0,86.0,61.0,0.0,27.56
2,178,White Non-Hispanic,65,"<$14,999",24.429492,1,1,1,Retired,2.0,...,2.0,18.0,0.0,59,58.0,67.0,73.0,61.0,0.0,27.78
3,178,White Non-Hispanic,65,"<$14,999",24.429492,1,1,1,Retired,2.0,...,2.0,18.0,0.0,60,56.0,68.0,76.0,60.0,0.0,27.81
4,178,White Non-Hispanic,65,"<$14,999",24.429492,1,1,1,Retired,2.0,...,2.0,18.0,0.0,61,56.0,64.0,77.0,51.0,0.0,27.89


In [14]:
# Extract the first obs for each individual
ID_idx = {}
for i, j in enumerate(data['ID']):
    ID_idx[j] = ID_idx.get(j, i)
Big = data.iloc[[value for (key, value) in sorted(ID_idx.items())],:]
Big.shape
Big.columns.values

array(['ID', 'Race', 'age', 'Income', 'bmi', 'treat', 'sex', 'nsaid',
       'Occupation', 'retire', 'agecat', 'inccat', 'racecat', 'opiate',
       'severe2', 'race2', 'pain.1', 'lastdt1', 'pain.2', 'lastdt2',
       'pain.3', 'lastdt3', 'pain.4', 'lastdt4', 'pain.5', 'lastdt5',
       'pain.6', 'lastdt6', 'pain.7', 'lastdt7', 'physi.1', 'stiff.1',
       'physi.4', 'stiff.4', 'physi.7', 'stiff.7', 'WeatherDate',
       'avgdewpt', 'avgtemp', 'maxtemp', 'mintemp', 'precip', 'press'], dtype=object)

In [15]:
# Predictors
predictors = ['age', 'treat', 'sex', 'nsaid', 'race2', 'inccat', 'Occupation']
pain_cols = np.array([16, 18, 20, 22, 24, 26, 28])
time_cols = pain_cols + 1

In [16]:
ids = Big["ID"]
dat = pd.DataFrame()

In [17]:
for i in range(len(ids)):
    id0 = ids.values[i]
    temp = data[data["ID"]==id0]
    zi = temp[["ID"]+predictors]
    pain = temp.iloc[0,pain_cols]
    pain = pain[pain.notnull()]
    time = temp.iloc[0,time_cols]
    temperature = temp["avgtemp"][[d in [k for k in time] for d in temp["WeatherDate"]]]
    time = time[time.notnull()]
    zi = zi[[d in [k for k in time] for d in temp["WeatherDate"]]]
    data_t = pd.concat([pd.concat([pain.reset_index(drop=True), time.reset_index(drop=True)],axis=1), temperature.reset_index(drop=True)], axis=1)
    data_t.columns = ["pain", "time", "temperature"]
    data_t = pd.concat([zi.reset_index(drop=True),data_t], axis=1)
    dat = pd.concat([dat,data_t], axis=0)

In [18]:
dat = dat[dat.iloc[:,0].notnull()]

In [19]:
dat.iloc[0:10,:]

,ID,age,treat,sex,nsaid,race2,inccat,Occupation,pain,time,temperature
0,178.0,65.0,1.0,1.0,1.0,1.0,1.0,Retired,7,57,70.0
1,178.0,65.0,1.0,1.0,1.0,1.0,1.0,Retired,6,99,56.0
2,178.0,65.0,1.0,1.0,1.0,1.0,1.0,Retired,8,113,48.0
3,178.0,65.0,1.0,1.0,1.0,1.0,1.0,Retired,8,127,35.0
4,178.0,65.0,1.0,1.0,1.0,1.0,1.0,Retired,7,141,36.0
0,180.0,59.0,0.0,2.0,1.0,1.0,3.0,RETIRED,16,3,67.0
1,180.0,59.0,0.0,2.0,1.0,1.0,3.0,RETIRED,7.5,18,65.0
2,180.0,59.0,0.0,2.0,1.0,1.0,3.0,RETIRED,7,32,65.0
3,180.0,59.0,0.0,2.0,1.0,1.0,3.0,RETIRED,12.5,45,78.0
4,180.0,59.0,0.0,2.0,1.0,1.0,3.0,RETIRED,6,59,58.0


In [20]:
# Change some types for each columns
dat["ID"] = dat["ID"].astype(int)
dat["age"] = dat["age"].astype(int)
dat["treat"] = dat["treat"].astype(int)
dat["sex"] = dat["sex"].astype(int)
dat["race2"] = dat["race2"].astype(int)
dat["pain"] = dat["pain"].astype(float)
dat["time"] = dat["time"].astype(int)

In [21]:
dat.iloc[0:10,:]

,ID,age,treat,sex,nsaid,race2,inccat,Occupation,pain,time,temperature
0,178,65,1,1,1.0,1,1.0,Retired,7.0,57,70.0
1,178,65,1,1,1.0,1,1.0,Retired,6.0,99,56.0
2,178,65,1,1,1.0,1,1.0,Retired,8.0,113,48.0
3,178,65,1,1,1.0,1,1.0,Retired,8.0,127,35.0
4,178,65,1,1,1.0,1,1.0,Retired,7.0,141,36.0
0,180,59,0,2,1.0,1,3.0,RETIRED,16.0,3,67.0
1,180,59,0,2,1.0,1,3.0,RETIRED,7.5,18,65.0
2,180,59,0,2,1.0,1,3.0,RETIRED,7.0,32,65.0
3,180,59,0,2,1.0,1,3.0,RETIRED,12.5,45,78.0
4,180,59,0,2,1.0,1,3.0,RETIRED,6.0,59,58.0


In [22]:
j = 0
for i in ids:
    if dat.iloc[j, 0] == i:
        target = dat.iloc[j, 9]
    while dat.iloc[j, 0] == i:
        dat.iloc[j, 9] -= target
        j += 1
        if j + 1 == dat.shape[0]:
            break
            


In [23]:
dat.head()

,ID,age,treat,sex,nsaid,race2,inccat,Occupation,pain,time,temperature
0,178,65,1,1,1.0,1,1.0,Retired,7.0,0,70.0
1,178,65,1,1,1.0,1,1.0,Retired,6.0,42,56.0
2,178,65,1,1,1.0,1,1.0,Retired,8.0,56,48.0
3,178,65,1,1,1.0,1,1.0,Retired,8.0,70,35.0
4,178,65,1,1,1.0,1,1.0,Retired,7.0,84,36.0


In [24]:
dat.to_csv('dat.csv', sep=',')